In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
dir = 'data/portfolio_decile_value'
port_lt = os.listdir(dir)
port = port_lt[0]

In [ ]:
def get_performance(port):
    file = dir + '/' + port
    df = pd.read_csv(file)
    decile_rows = df[df['Unnamed: 0'] == 'decile']
    decile_values = decile_rows['0'].tolist()

    output_df = pd.DataFrame()
    for i, value in enumerate(decile_values):
        if i == 0:
            decile_df = df.iloc[:decile_rows.index[i]]
            decile_df.columns = ['Date', str(int(value))]
            output_df = decile_df.copy(deep=True)
        elif i == len(decile_values)-1:
            decile_df = df.iloc[decile_rows.index[i-1]+1:]
            decile_df.columns = ['Date', str(int(value))]
            output_df = pd.merge(output_df, decile_df, how='left', on='Date')
        else:
            decile_df = df.iloc[decile_rows.index[i-1]+1:decile_rows.index[i]]
            decile_df.columns = ['Date', str(int(value))]
            output_df = pd.merge(output_df, decile_df, how='left', on='Date')
    new_df = pd.DataFrame({'Date': '2019-02-28','1':1,'2':1,'3':1,'4':1,'5':1,'6':1,'7':1,'8':1,'9':1,'10':1}, index=[0])
    output_df = pd.concat([new_df,output_df], ignore_index=True)
    output_df.set_index('Date', inplace=True)
    output_df = output_df.pct_change().dropna()
    output_df = output_df.loc[:,(output_df != 0).any(axis=0)]
    #print(output_df)
    mean = (output_df.mean() + 1)**252 - 1
    std = output_df.std() * np.sqrt(252)
    ff = pd.read_csv('data/F-F_Research_Data_Factors_daily.CSV')
    ff.columns = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RF']
    ff['Date'] = pd.to_datetime(ff['Date'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
    ff.set_index('Date', inplace=True)
    output_df = pd.merge(output_df, ff['RF']/100, how='left', on='Date')
    mean_rf = (output_df['RF'].mean() + 1)**252 - 1
    sr = (mean - mean_rf) / std
    return mean, std, sr

In [ ]:
os.makedirs('data/portfolio_decile_vis', exist_ok=True)
def vis_performance(port,mean, std, sr):
    fig, axes = plt.subplots(1, 3, figsize=(15, 6))
    axes[0].bar(mean.index[::-1], mean)
    axes[0].set_title('Mean Return')
    axes[1].bar(std.index[::-1], std)
    axes[1].set_title('Standard Deviation')
    axes[2].bar(sr.index[::-1], sr)
    axes[2].set_title('Sharpe Ratio')
    fig.suptitle(f"{port.split('.')[0].split('_')[-1]} (annualized)", fontsize=16)
    fig.savefig('data/portfolio_decile_vis/' + port + '.png')
    plt.close(fig)

In [ ]:
for port in port_lt:
    mean, std, sr = get_performance(port)
    vis_performance(port, mean, std, sr)